## preprocess

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import torch
from torch_geometric.data import Data
import os
import torch.nn.functional as F
import pickle
import json
import warnings
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
warnings.filterwarnings('ignore')
from torch_geometric.loader import NeighborLoader
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from collections import Counter
from gensim.models import Word2Vec
from multiprocessing import Pool
from itertools import compress
from tqdm import tqdm
import time
import torch
torch.cuda.is_available()  # 如果返回True，说明PyTorch可以看到GPU
torch.cuda.device_count()  # 返回GPU的数量
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import os
import json
import psutil
import re
import pandas as pd
import numpy as np
import torch
from torch_geometric import utils
from torch_geometric.loader import NeighborLoader
from torch_geometric.data import Data


# # 获取当前进程，设置进程亲和性（假设你想使用CPU 17）
p = psutil.Process(os.getpid())
p.cpu_affinity([16])

%matplotlib inline

/home/cch/.conda/envs/cch_TFE_GNN/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import hashlib
def stringtomd5(originstr):
    originstr = originstr.encode("utf-8")
    signaturemd5 = hashlib.sha256()
    signaturemd5.update(originstr)
    return signaturemd5.hexdigest()

In [3]:


# /home/cch/TraceSentinel/data/optc/benign/20-23Sep19/concat_SysClient0051.systemia.com.txt
with open("../data/optc/evaluation/25Sept/AIA-51-75/SysClient0051.systemia.com.txt", 'r') as file:
    content = [json.loads(line) for line in file]




def Extract_Semantic_Info(event):
    object_type = event['object']
    properties = event['properties']

    label_mapping = {
        "PROCESS": ('parent_image_path', 'image_path'),
        "FILE": ('image_path', 'file_path'),
        "FLOW": ('image_path', 'dest_ip', 'dest_port', 'direction'),
        'MODULE': ('image_path', 'module_path')
    }

    label_keys = label_mapping.get(object_type, None)
    if label_keys:
        labels = [properties.get(key) for key in label_keys]
        # print(labels)
        if all(labels):
            if object_type=="PROCESS":
                event["actorname"], event["objectname"] = labels[0], labels[1]
                event["command_line"] = properties.get("command_line", "None") if len(properties.get("command_line", "None")) > 70 else properties.get("command_line", "None")
                # event["command_line"] = labels[2] if len(labels[2]) > 70 else labels[2]
            elif object_type=="FILE":
                event["actorname"], event["objectname"] = labels[0], labels[1]
            elif object_type=="FLOW":
                event["actorname"], event["objectname"] = labels[0], '-'.join(labels[1:3])
                event['action'] = labels[3]
            elif object_type == "MODULE":
                event["actorname"], event["objectname"] = labels[0], labels[1]
            else:
                pass
            return event
    return None


def Sentence_Construction(entry):
    action = entry["action"]
    properties = entry['properties']
    object_type = entry['object']
    phrase=[]
    try:
        if object_type=="PROCESS":
            phrase.append(properties.get("parent_image_path", "N/A"))
            phrase.append(action)
            phrase.append(properties.get("image_path", "N/A"))
            # phrase.append(properties.get("command_line", "N/A"))
        elif object_type=="FILE":
            phrase.append(properties.get("image_path", "N/A"))
            phrase.append(action)
            phrase.append(properties.get("file_path", "N/A"))
        elif object_type=="FLOW":
            phrase.append(properties.get("image_path", "N/A"))
            phrase.append(action)
            dest_ip = str(properties.get("dest_ip", "N/A"))
            dest_port = str(properties.get("dest_port", "N/A"))
            phrase.append(dest_ip+'-'+dest_port)
            # phrase.append(properties.get("src_ip", "N/A"))
            # phrase.append(properties.get("src_port", "N/A"))
            # phrase.append(properties.get("dest_ip", "N/A"))
            # phrase.append(properties.get("dest_port", "N/A"))
            # phrase.append(properties.get("direction", "N/A"))
        elif object_type=="MODULE":
            phrase.append(properties.get("image_path", "N/A"))
            phrase.append(action)
            phrase.append(properties.get("module_path", "N/A"))
        else:
            pass
    except KeyError:
        phrase = []

    return phrase

def is_valid_entry(entry):
    valid_objects = {'PROCESS', 'FILE', 'FLOW', 'MODULE'}
    invalid_actions = {'START', 'TERMINATE', 'OPEN'}

    object_valid = entry['object'] in valid_objects
    action_valid = entry['action'] not in invalid_actions
    actor_object_different = entry['actorID'] != entry['objectID']

    return object_valid and action_valid and actor_object_different

def Traversal_Rules(data):
    filtered_data = []

    for entry in data:
        if is_valid_entry(entry):
            filtered_data.append(entry)

    return filtered_data


def transform(text):
    labeled_data = [event for event in (Extract_Semantic_Info(x) for x in text) if event]
    data = Traversal_Rules(labeled_data)


    phrases = [Sentence_Construction(x) for x in data if Sentence_Construction(x)]
    for datum, phrase in zip(data, phrases):
        datum['phrase'] = phrase

    # data = sorted(data, key=lambda x: x.get('timestamp'))

    df = pd.DataFrame(data)
    df['timestamp'] = pd.to_datetime(df['timestamp'].str[:19], format='%Y-%m-%dT%H:%M:%S')
    df.sort_values(by='timestamp', inplace=True)
    df = df.reset_index(drop=True)

    return df


df = transform(content)
df.head()

,action,actorID,hostname,id,object,objectID,pid,ppid,principal,properties,tid,timestamp,actorname,objectname,command_line,phrase
0,inbound,dfcbb954-fb85-41a3-8575-9af61053f01c,SysClient0051.systemia.com,185902d5-4ad9-4071-8709-d0922a1c66bd,FLOW,2b0bdbf8-ff67-47d9-9ff3-8cea315acf7e,876,548,NT AUTHORITY\NETWORK SERVICE,"{'acuity_level': '4', 'dest_ip': '224.0.0.252'...",-1,2019-09-25 09:04:44,\Device\HarddiskVolume1\Windows\system32\svcho...,224.0.0.252-5355,NaN,[\Device\HarddiskVolume1\Windows\system32\svch...
1,inbound,dfcbb954-fb85-41a3-8575-9af61053f01c,SysClient0051.systemia.com,c1cb1ba5-7dcf-46c0-9563-0b3ba02b6318,FLOW,817dd001-3560-422e-a6b6-13d91b391122,876,548,NT AUTHORITY\NETWORK SERVICE,"{'acuity_level': '4', 'dest_ip': '224.0.0.252'...",-1,2019-09-25 09:04:44,\Device\HarddiskVolume1\Windows\system32\svcho...,224.0.0.252-5355,NaN,[\Device\HarddiskVolume1\Windows\system32\svch...
2,inbound,dfcbb954-fb85-41a3-8575-9af61053f01c,SysClient0051.systemia.com,868c369a-ad9e-4dd0-9b57-080afd127123,FLOW,f3f21c9d-7842-49eb-9af9-bb05a9839448,876,548,NT AUTHORITY\NETWORK SERVICE,"{'acuity_level': '4', 'dest_ip': '224.0.0.252'...",-1,2019-09-25 09:04:44,\Device\HarddiskVolume1\Windows\system32\svcho...,224.0.0.252-5355,NaN,[\Device\HarddiskVolume1\Windows\system32\svch...
3,inbound,dfcbb954-fb85-41a3-8575-9af61053f01c,SysClient0051.systemia.com,f54d85d3-681c-49e7-80b1-1371307d62cc,FLOW,0a038618-dae9-46a0-b3ab-f1c9cd928ef4,876,548,NT AUTHORITY\NETWORK SERVICE,"{'acuity_level': '4', 'dest_ip': '224.0.0.252'...",-1,2019-09-25 09:04:44,\Device\HarddiskVolume1\Windows\system32\svcho...,224.0.0.252-5355,NaN,[\Device\HarddiskVolume1\Windows\system32\svch...
4,inbound,dfcbb954-fb85-41a3-8575-9af61053f01c,SysClient0051.systemia.com,fd7f1f38-7de7-440d-95b9-e569841b845c,FLOW,6e56dc1d-0c7f-420f-9ae2-766afd2743a5,876,548,NT AUTHORITY\NETWORK SERVICE,"{'acuity_level': '4', 'dest_ip': '224.0.0.252'...",-1,2019-09-25 09:04:44,\Device\HarddiskVolume1\Windows\system32\svcho...,224.0.0.252-5355,NaN,[\Device\HarddiskVolume1\Windows\system32\svch...


Series([], Name: action, dtype: int64)

In [4]:
start_time = '2019-09-25 9:30:00'
end_time = '2019-09-25 15:00:00'
df = df[(df['timestamp'] > start_time) & (df['timestamp'] < end_time)]

In [5]:
# df[(df['object'] == 'process') & (df['actorname'] == 'system')]

In [5]:
def proprocess_df(df):
    # 指定列转换为小写
    columns_to_lower = ['action', 'object', 'actorname', 'objectname', 'command_line', 'phrase']
    # df[columns_to_lower] = df[columns_to_lower].applymap(lambda x: x.lower() if isinstance(x, str) else x)
    # 使用apply和lambda函数转换列中的值
    df[columns_to_lower] = df[columns_to_lower].applymap(lambda x: x.lower() if isinstance(x, str) else x)
    # 对'phrase'列中的每个元素（列表中的字符串）应用小写转换
    df['phrase'] = df['phrase'].apply(lambda x: [item.lower() if isinstance(item, str) else item for item in x])

    node2name = {}
    # 首先遍历create事件，提取进程名
    extract_process_name = df[(df['object'] == 'process') & (df['action'] == 'create')]
    for row in extract_process_name.itertuples():
        nodeid = row.objectID
        processname = row.objectname
        if nodeid not in node2name:
            node2name[nodeid] = processname
    for row in df[(df['object'] == 'process') & (df['action'] != 'create')].itertuples():
        nodeid = row.objectID
        processname = row.objectname
        if nodeid not in node2name:
            node2name[nodeid] = processname
    # 使用 map 方法将 node2name 字典中的值应用到objectname列
    # fillna(df['objectname']) 保证如果没有对应的 nodeid，在不改变原来的值的情况下保持 objectname 原样
    df['objectname'] = df['objectID'].map(node2name).fillna(df['objectname'])
    df['actorname'] = df['actorID'].map(node2name).fillna(df['actorname'])


    new_path = {}
    for row in df[(df['object'] == 'file') & (df['action'] == 'rename')].itertuples():
        nodeid = row.objectID
        new_pathname = row.properties.get('new_path', 'None')
        if nodeid not in new_path:
            new_path[nodeid] = new_pathname

    df['objectname'] = df['objectID'].map(new_path).fillna(df['objectname'])

    # 去除重复事件
    df.drop_duplicates(subset=['action', 'actorID', 'objectID', 'object', 'actorname', 'objectname', 'pid', 'ppid'], inplace=True)
    df = df.reset_index(drop=True)

    # 根据进程名称和pid生成新的索引
    for index, row in df.iterrows():
        if row['object'] == 'process':
            # 对于 'process' 类型
            src_id = stringtomd5(row['actorname'] + str(row['ppid']))
            dest_id = stringtomd5(row['objectname'] + str(row['pid']))
        elif row['object'] in ['flow', 'file', 'module']:
            # 对于 'flow' 或 'file' 类型
            src_id = stringtomd5(row['actorname'] + str(row['pid']))
            dest_id = stringtomd5(row['objectname'])
        else:
            # 如果其他类型的 object
            src_id = None
            dest_id = None
        
        # 将结果存入新的列
        df.at[index, 'src_id'] = src_id
        df.at[index, 'dest_id'] = dest_id

    # 去除重复事件
    df.drop_duplicates(subset=['action', 'src_id', 'dest_id', 'object', 'actorname', 'objectname'], inplace=True)
    df = df[df['src_id'] != df['dest_id']]
    df = df.reset_index(drop=True)
    return df

In [6]:
df = proprocess_df(df)

In [7]:
df.head()

,action,actorID,hostname,id,object,objectID,pid,ppid,principal,properties,tid,timestamp,actorname,objectname,command_line,phrase,src_id,dest_id
0,delete,ef72c3a6-b587-4b9d-b9bd-87952b524613,SysClient0051.systemia.com,5bf730d9-5a56-4c6c-8cd8-4a1bc3568871,file,b16f62fd-19a6-43ea-9792-98a9eb953086,3792,404,SYSTEMIACOM\dcoombes,"{'acuity_level': '4', 'file_path': '\Device\Ha...",5208,2019-09-25 09:30:01,\device\harddiskvolume1\windows\system32\clean...,\device\harddiskvolume1\users\dcoombes\appdata...,NaN,[\device\harddiskvolume1\windows\system32\clea...,a111e40ba4963e47ab7313431ab6599121d067826f067e...,dbdaaaab65b1c4ded54a4c63a74205d38a713b5a96a15a...
1,delete,ef72c3a6-b587-4b9d-b9bd-87952b524613,SysClient0051.systemia.com,29b36464-3182-4796-9d8a-1505bd10d241,file,870f7f75-fc3f-44ab-98db-862133ed5033,3792,404,SYSTEMIACOM\dcoombes,"{'acuity_level': '4', 'file_path': '\Device\Ha...",5208,2019-09-25 09:30:01,\device\harddiskvolume1\windows\system32\clean...,\device\harddiskvolume1\users\dcoombes\appdata...,NaN,[\device\harddiskvolume1\windows\system32\clea...,a111e40ba4963e47ab7313431ab6599121d067826f067e...,5b9de639e4c0037494877f625656384aa87588a04bbe15...
2,delete,ef72c3a6-b587-4b9d-b9bd-87952b524613,SysClient0051.systemia.com,a18d7b44-974b-4772-8cd0-cfa14be1d5d9,file,c31be0be-64a0-416a-a0a2-c779fd15c8f1,3792,404,SYSTEMIACOM\dcoombes,"{'acuity_level': '4', 'file_path': '\Device\Ha...",5208,2019-09-25 09:30:01,\device\harddiskvolume1\windows\system32\clean...,\device\harddiskvolume1\users\dcoombes\appdata...,NaN,[\device\harddiskvolume1\windows\system32\clea...,a111e40ba4963e47ab7313431ab6599121d067826f067e...,5d094d1ec8f89d39830eef15a7a2588dd47bcb5d95f094...
3,delete,ef72c3a6-b587-4b9d-b9bd-87952b524613,SysClient0051.systemia.com,d8c5677a-db39-42a1-bcfa-73e216a522b7,file,18e9ffcf-1e32-4a0a-8e80-27bf7f30e1ed,3792,404,SYSTEMIACOM\dcoombes,"{'acuity_level': '4', 'file_path': '\Device\Ha...",5208,2019-09-25 09:30:01,\device\harddiskvolume1\windows\system32\clean...,\device\harddiskvolume1\users\dcoombes\appdata...,NaN,[\device\harddiskvolume1\windows\system32\clea...,a111e40ba4963e47ab7313431ab6599121d067826f067e...,1888033a4f0ce9df8ea7b947cdc3a591ff5638e1073957...
4,delete,ef72c3a6-b587-4b9d-b9bd-87952b524613,SysClient0051.systemia.com,36dc480c-2352-489c-841b-326cfc558a6d,file,527db162-19a7-4565-9cd7-7e98eeaca25f,3792,404,SYSTEMIACOM\dcoombes,"{'acuity_level': '4', 'file_path': '\Device\Ha...",5208,2019-09-25 09:30:01,\device\harddiskvolume1\windows\system32\clean...,\device\harddiskvolume1\users\dcoombes\appdata...,NaN,[\device\harddiskvolume1\windows\system32\clea...,a111e40ba4963e47ab7313431ab6599121d067826f067e...,79cfb5d0759235f337a41dcd55600ecc2ef546370ad435...


In [8]:
df[df['object'] == 'process'].head()

,action,actorID,hostname,id,object,objectID,pid,ppid,principal,properties,tid,timestamp,actorname,objectname,command_line,phrase,src_id,dest_id
229,create,eca905c9-9bae-4053-8c01-25c136a6e2f2,SysClient0051.systemia.com,184459d6-9fe5-497a-965f-bb53484659df,process,f1903b83-7779-4e0a-bdd8-01798aa4150b,3844,3552,NT AUTHORITY\SYSTEM,"{'acuity_level': '1', 'command_line': '\??\C:\...",-1,2019-09-25 09:30:01,\device\harddiskvolume1\windows\system32\conho...,\device\harddiskvolume1\windows\system32\conho...,\??\c:\windows\system32\conhost.exe 0xffffffff...,[\device\harddiskvolume1\windows\system32\conh...,bd78ea3bdfc75490bb5e95f55ad4fad564e9e43cd99fb9...,02ae087718698097cb2253a86427c0a8ccb1d9fe6c6ded...
230,create,326c1a19-9193-48fb-868a-66dc051623f4,SysClient0051.systemia.com,8102c713-d599-45d9-bea4-d0d9d0924142,process,377feb67-195e-4521-b5ec-e70c5d86ddb9,3552,404,NT AUTHORITY\SYSTEM,"{'acuity_level': '1', 'command_line': 'C:\Wind...",-1,2019-09-25 09:30:01,\device\harddiskvolume1\windows\system32\svcho...,\device\harddiskvolume1\windows\system32\cmd.exe,"c:\windows\system32\cmd.exe /c ""c:\ncr\deletea...",[\device\harddiskvolume1\windows\system32\svch...,699979adefa4b3f1be8121f2c8b5a3669f9bd2e1749cd6...,eefb7d17c3858e24f0899236231425cba30c3fb3daaa09...
388,create,326c1a19-9193-48fb-868a-66dc051623f4,SysClient0051.systemia.com,6386d242-99c2-40f0-b958-c41b51550017,process,78f38caf-5faa-4ed7-8b2c-9710b9267c1f,5476,404,NT AUTHORITY\SYSTEM,"{'acuity_level': '1', 'command_line': 'taskhos...",-1,2019-09-25 09:30:03,\device\harddiskvolume1\windows\system32\svcho...,taskhostw.exe,taskhostw.exe,[\device\harddiskvolume1\windows\system32\svch...,699979adefa4b3f1be8121f2c8b5a3669f9bd2e1749cd6...,02c0251865cb38fa7fafeb6d4bc3cce4cb763862739c43...
671,create,99480f4c-aea3-4ae8-ba4d-3aabdf270cb9,SysClient0051.systemia.com,d94f9d46-0747-4f0e-b670-2c9d7b970672,process,d333d859-d7aa-4c79-8101-a80f87b7bcd7,5360,2192,NT AUTHORITY\SYSTEM,"{'acuity_level': '1', 'command_line': 'C:\Wind...",-1,2019-09-25 09:30:53,\device\harddiskvolume1\python27\python.exe,\device\harddiskvolume1\windows\system32\cmd.exe,"c:\windows\system32\cmd.exe /c ""powershell -c ...","[\device\harddiskvolume1\python27\python.exe, ...",7c865490e119e47808408788ab23557b89d06ea73c507e...,75ea8c918ee1597e538436eae731ef8c71de96a4430b8c...
694,create,d333d859-d7aa-4c79-8101-a80f87b7bcd7,SysClient0051.systemia.com,fe84787d-c200-4f92-ab03-b534369128db,process,186c0097-f69c-46f5-8f9c-7ce932ddb6b2,4556,5360,NT AUTHORITY\SYSTEM,"{'acuity_level': '1', 'command_line': 'powersh...",-1,2019-09-25 09:30:53,\device\harddiskvolume1\windows\system32\cmd.exe,powershell.exe,"powershell -c ""start-service lwabeat""",[\device\harddiskvolume1\windows\system32\cmd....,75ea8c918ee1597e538436eae731ef8c71de96a4430b8c...,22bdc0262ff338f44c5462b40ef4e5b63ec99f333df37a...


In [15]:
# df[(df['object'] == 'file') & (df['action'] == 'create')].head()

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64971 entries, 0 to 64970
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   action        64971 non-null  object        
 1   actorID       64971 non-null  object        
 2   hostname      64971 non-null  object        
 3   id            64971 non-null  object        
 4   object        64971 non-null  object        
 5   objectID      64971 non-null  object        
 6   pid           64971 non-null  int64         
 7   ppid          64971 non-null  int64         
 8   principal     64971 non-null  object        
 9   properties    64971 non-null  object        
 10  tid           64971 non-null  int64         
 11  timestamp     64971 non-null  datetime64[ns]
 12  actorname     64971 non-null  object        
 13  objectname    64971 non-null  object        
 14  command_line  1158 non-null   object        
 15  phrase        64971 non-null  object

In [10]:
df = df[df['object'] != "module"]
read_df = df[df['action'] == 'read']
write_actions = ['write', 'modify', 'create', 'rename']

# 筛选出写操作的行，并排除由 system 和 dwm.exe 修改的文件
# write_df = df[(df['action'].isin(write_actions)) & (~df['actorname'].isin(['system', 'dwm.exe']))]
sys_dwm = df[df['actorname'].isin(['system', 'dwm.exe'])]['dest_id']

write_df = df[df['action'].isin(write_actions)]



# write_object_ids = set(write_df['dest_id'])
write_object_ids = set(write_df['dest_id']) - set(sys_dwm)
readonly_df = read_df[~read_df['dest_id'].isin(write_object_ids)]
readonly_df_ids = readonly_df['dest_id']
df = df[~df['dest_id'].isin(readonly_df_ids)]

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11478 entries, 0 to 64940
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   action        11478 non-null  object        
 1   actorID       11478 non-null  object        
 2   hostname      11478 non-null  object        
 3   id            11478 non-null  object        
 4   object        11478 non-null  object        
 5   objectID      11478 non-null  object        
 6   pid           11478 non-null  int64         
 7   ppid          11478 non-null  int64         
 8   principal     11478 non-null  object        
 9   properties    11478 non-null  object        
 10  tid           11478 non-null  int64         
 11  timestamp     11478 non-null  datetime64[ns]
 12  actorname     11478 non-null  object        
 13  objectname    11478 non-null  object        
 14  command_line  1158 non-null   object        
 15  phrase        11478 non-null  object

In [10]:
# df[df['objectname']=="\device\harddiskvolume1\windows\system32\devobj.dll"]

In [12]:
df.tail()

,action,actorID,hostname,id,object,objectID,pid,ppid,principal,properties,tid,timestamp,actorname,objectname,command_line,phrase,src_id,dest_id
64864,create,854ac0d9-a8e9-4ccf-8837-0a283e30ece8,SysClient0051.systemia.com,11d2a482-e99d-490d-89e9-2c9ce24f3b07,process,803becd8-f8a7-4d2f-ada9-70da37bd80b8,3160,400,NT AUTHORITY\SYSTEM,"{'acuity_level': '1', 'command_line': 'C:\Wind...",-1,2019-09-25 14:37:00,\device\harddiskvolume1\windows\system32\svcho...,\device\harddiskvolume1\windows\system32\cmd.exe,"c:\windows\system32\cmd.exe /c ""c:\ncr\deletea...",[\device\harddiskvolume1\windows\system32\svch...,e705b16796d218e4fd7c654a1592cbcf1c51637422801b...,7b38287ed3f294572686bda3ae8464d785885fa63d0b8d...
64889,create,803becd8-f8a7-4d2f-ada9-70da37bd80b8,SysClient0051.systemia.com,5d48690b-7ae4-40f8-9ae4-775eed169a4b,process,dc55f28c-1140-4850-a9ab-83703d0c62d0,2180,3160,NT AUTHORITY\SYSTEM,"{'acuity_level': '1', 'command_line': '\??\C:\...",-1,2019-09-25 14:37:00,\device\harddiskvolume1\windows\system32\cmd.exe,\device\harddiskvolume1\windows\system32\conho...,\??\c:\windows\system32\conhost.exe 0xffffffff...,[\device\harddiskvolume1\windows\system32\cmd....,7b38287ed3f294572686bda3ae8464d785885fa63d0b8d...,e180542bfd654d3dc348633f7fbf48f7270e7bad6338e8...
64914,inbound,bda7d275-8bbf-4c5b-8130-10a95b829a3e,SysClient0051.systemia.com,194d5b7f-493c-4cbd-b450-fb0b0a5b75f2,flow,3f5fbc92-c34c-4ef5-a49e-9a10430a0a01,664,564,NT AUTHORITY\LOCAL SERVICE,"{'acuity_level': '4', 'dest_ip': '142.20.56.52...",-1,2019-09-25 14:37:21,\device\harddiskvolume1\windows\system32\svcho...,142.20.56.52-123,NaN,[\device\harddiskvolume1\windows\system32\svch...,abe34e9aeabc4f94fa2543c9af1daf07f0e82bac446b16...,0221686c869ab5d48f97301ae058a43280148cca88dd47...
64939,create,7146c5ef-254a-4450-b132-ce6840948a8b,SysClient0051.systemia.com,10feffc5-d353-4872-ac92-3da8bbe3380d,process,f2a206e1-1b79-4f18-a36c-89812fa1df95,4044,3404,NT AUTHORITY\SYSTEM,"{'acuity_level': '1', 'command_line': '\??\C:\...",-1,2019-09-25 14:38:00,\device\harddiskvolume1\windows\system32\cmd.exe,\device\harddiskvolume1\windows\system32\conho...,\??\c:\windows\system32\conhost.exe 0xffffffff...,[\device\harddiskvolume1\windows\system32\cmd....,2b556ee50f47a0569028ccf2929e6f8cb2e4a661f826c8...,5f299ebd22e79488adf703b05a2831bbae0139e1b83959...
64940,create,854ac0d9-a8e9-4ccf-8837-0a283e30ece8,SysClient0051.systemia.com,9e9df33e-7c72-4fbe-81ea-877c87791dee,process,7146c5ef-254a-4450-b132-ce6840948a8b,3404,400,NT AUTHORITY\SYSTEM,"{'acuity_level': '1', 'command_line': 'C:\Wind...",-1,2019-09-25 14:38:00,\device\harddiskvolume1\windows\system32\svcho...,\device\harddiskvolume1\windows\system32\cmd.exe,"c:\windows\system32\cmd.exe /c ""c:\ncr\deletea...",[\device\harddiskvolume1\windows\system32\svch...,e705b16796d218e4fd7c654a1592cbcf1c51637422801b...,2b556ee50f47a0569028ccf2929e6f8cb2e4a661f826c8...


In [13]:
# 从df中去除名为\device\harddiskvolume1\windows\system32\svchost.exe和\device\harddiskvolume1\windows\system32\conhost.exe的节点
exclude_values = [
    '\device\harddiskvolume1\windows\system32\svchost.exe',
    '\device\harddiskvolume1\windows\system32\conhost.exe',
    'svchost.exe',
    'consent.exe'
]

# 过滤掉包含指定值的行
df = df[~df['actorname'].isin(exclude_values) & ~df['objectname'].isin(exclude_values)]
df = df.reset_index(drop=True)

In [14]:
# 找到所有 create 操作的父进程和子进程
parent_child_df = df[(df['action'] == 'create') & (df['object'] == 'process')]

# 提取所有父进程和子进程的 ID
parent_ids = set(parent_child_df['src_id'])  # 父进程 ID
child_ids = set(parent_child_df['dest_id'])   # 子进程 ID

# 所有相关的进程 ID（即有父进程或有子进程的）
related_ids = parent_ids.union(child_ids)

# 从 df 中移除既没有父进程也没有子进程的节点
df = df[df['src_id'].isin(related_ids) | df['dest_id'].isin(related_ids)]

# 重置索引
df = df.reset_index(drop=True)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5421 entries, 0 to 5420
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   action        5421 non-null   object        
 1   actorID       5421 non-null   object        
 2   hostname      5421 non-null   object        
 3   id            5421 non-null   object        
 4   object        5421 non-null   object        
 5   objectID      5421 non-null   object        
 6   pid           5421 non-null   int64         
 7   ppid          5421 non-null   int64         
 8   principal     5421 non-null   object        
 9   properties    5421 non-null   object        
 10  tid           5421 non-null   int64         
 11  timestamp     5421 non-null   datetime64[ns]
 12  actorname     5421 non-null   object        
 13  objectname    5421 non-null   object        
 14  command_line  394 non-null    object        
 15  phrase        5421 non-null   object  

In [16]:
# 过滤掉没有cmdline的进程节点：
df = df[~((df['object'] == 'process') & (df['command_line'] == 'none'))]

In [17]:
import pandas as pd
import networkx as nx

# 假设 df 是你的 DataFrame
# 1. 筛选 object 列为 'process' 的行
df_process = df[df['object'] == 'process']

# 2. 创建一个无向图
G = nx.Graph()

# 3. 添加边 (src_id, dest_id) 到图中
for _, row in df_process.iterrows():
    G.add_edge(row['src_id'], row['dest_id'])

# 4. 获取图中的所有连通子图
connected_components = list(nx.connected_components(G))

# 5. 过滤掉节点数小于 2 的子图
filtered_components = [comp for comp in connected_components if len(comp) > 2]

# 6. 获取所有保留的节点
valid_nodes = set(node for component in filtered_components for node in component)

# 7. 根据 valid_nodes 过滤原 DataFrame 中的行
# df_filtered = df_process[df_process['src_id'].isin(valid_nodes) & df_process['dest_id'].isin(valid_nodes)]

# 结果：df_filtered 就是筛选后的 DataFrame
df = df[(df['src_id'].isin(valid_nodes) | df['dest_id'].isin(valid_nodes))]

In [333]:
# filtered_components

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4590 entries, 0 to 5420
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   action        4590 non-null   object        
 1   actorID       4590 non-null   object        
 2   hostname      4590 non-null   object        
 3   id            4590 non-null   object        
 4   object        4590 non-null   object        
 5   objectID      4590 non-null   object        
 6   pid           4590 non-null   int64         
 7   ppid          4590 non-null   int64         
 8   principal     4590 non-null   object        
 9   properties    4590 non-null   object        
 10  tid           4590 non-null   int64         
 11  timestamp     4590 non-null   datetime64[ns]
 12  actorname     4590 non-null   object        
 13  objectname    4590 non-null   object        
 14  command_line  384 non-null    object        
 15  phrase        4590 non-null   object  

In [19]:
df[(df['src_id']=='255baa7d3044f87213df50991bc4999b83bcb83df8de4d4859a5698d08e92184') & (df['object'] == 'process')]

,action,actorID,hostname,id,object,objectID,pid,ppid,principal,properties,tid,timestamp,actorname,objectname,command_line,phrase,src_id,dest_id
4976,create,e449f056-94a3-4bce-aae7-70b9d2cdba62,SysClient0051.systemia.com,179be642-a51e-4d17-a169-080264849d8b,process,546dc19f-2415-4d43-a288-acd015b05f9c,292,4,NT AUTHORITY\SYSTEM,"{'acuity_level': '1', 'command_line': '\System...",-1,2019-09-25 14:21:31,system,\systemroot\system32\smss.exe,\systemroot\system32\smss.exe,"[system, create, \systemroot\system32\smss.exe]",255baa7d3044f87213df50991bc4999b83bcb83df8de4d...,45e1c507ad85a24d4b5a02585eb5a868f6b6e0be84a05d...


In [20]:
df['action'].value_counts()

create      1400
modify      1250
write        811
read         507
inbound      267
delete       227
outbound     106
rename        22
Name: action, dtype: int64

In [173]:
# ## 增加过滤规则，过滤掉符合如下条件的节点，我们认为这个是噪声节点：前向循环查找和后向循环查找都只有一跳。
# # 筛选 df 中 object 为 'process' 的行
# df_process = df[df['object'] == 'process']

# # 筛选规则 1: 没有子进程，且父进程没有父进程
# # 通过 src_id 和 dest_id 来过滤，确保父进程没有父进程
# # 以dest_id为中心进行处理
# no_children_and_no_grandparent = df_process[
#     (~df_process['dest_id'].isin(df_process['src_id'])) &  # 其为父进程                       dest_id是有子进程的
#     (~df_process['src_id'].isin(df_process['dest_id']))  # 父进程没有父进程                   
# ]

# # 筛选规则 2: 没有父进程，仅有子进程，且其子进程没有子进程
# # 通过 src_id 和 dest_id 来过滤，确保没有父进程，仅有子进程，且子进程没有进一步子进程
# # 以src_id为中心进行处理
# no_parent_with_children_and_no_grandchildren = df_process[
#     (df_process['src_id'].isin(df_process['dest_id'])) &  # 没有父进程
#     (~df_process['dest_id'].isin(df_process['src_id']))  # 子进程没有进一步子进程
# ]

# # 合并两种规则，去除符合条件的进程
# filtered_df = df_process[(df_process.index.isin(no_children_and_no_grandparent.index) | 
#                          df_process.index.isin(no_parent_with_children_and_no_grandchildren.index))]

# # 将筛选结果与原数据框合并
# df = df[~df.index.isin(filtered_df.index)]

# # 重置索引
# df = df.reset_index(drop=True)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4590 entries, 0 to 5420
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   action        4590 non-null   object        
 1   actorID       4590 non-null   object        
 2   hostname      4590 non-null   object        
 3   id            4590 non-null   object        
 4   object        4590 non-null   object        
 5   objectID      4590 non-null   object        
 6   pid           4590 non-null   int64         
 7   ppid          4590 non-null   int64         
 8   principal     4590 non-null   object        
 9   properties    4590 non-null   object        
 10  tid           4590 non-null   int64         
 11  timestamp     4590 non-null   datetime64[ns]
 12  actorname     4590 non-null   object        
 13  objectname    4590 non-null   object        
 14  command_line  384 non-null    object        
 15  phrase        4590 non-null   object  

In [22]:
# df[df['src_id'] == "30baed40b3a827e8014db1360e718a6689cbbdc47a65262db4d760ab742e3cd8"]
df[df['actorname'] == 'idle']

,action,actorID,hostname,id,object,objectID,pid,ppid,principal,properties,tid,timestamp,actorname,objectname,command_line,phrase,src_id,dest_id


## word2vec

In [23]:
import json
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec

class EpochSaver(CallbackAny2Vec):

    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        model.save('../trained_weights/optc/cmdline_attack3_w2v.model')
        self.epoch += 1

class EpochLogger(CallbackAny2Vec):

    def __init__(self):
        self.epoch = 0

    def on_epoch_begin(self, model):
        print("Epoch #{} start".format(self.epoch))

    def on_epoch_end(self, model):
        print("Epoch #{} end".format(self.epoch))
        self.epoch += 1

def prepare_sentences(df):
    nodes = {}
    for index, row in df.iterrows():
        for key in ['src_id', 'dest_id']:
            node_id = row[key]
            nodes.setdefault(node_id, []).extend(row['phrase'])
    return list(nodes.values())

def train_word2vec_model(df):
    phrases = prepare_sentences(df)

    logger = EpochLogger()
    saver = EpochSaver()
    word2vec = Word2Vec(sentences=phrases, vector_size=64, window=5, min_count=1, workers=8, epochs=50, callbacks=[saver, logger])

    return word2vec

In [9]:
# train_word2vec_model(df)

In [24]:
# 训练过程见optc_parser3
word2vec_weights = '../trained_weights/optc/cmdline_attack3_w2v.model'
w2vmodel = Word2Vec.load(word2vec_weights)

## featurize

In [25]:
import numpy as np


def infer(word):
    if word in  w2vmodel.wv:
        word_embeddings = w2vmodel.wv[word]
    else:
        word_embeddings = np.zeros(64)

    output_embedding = torch.tensor(word_embeddings, dtype=torch.float)
    output_embedding = output_embedding.detach().cpu().numpy()
    return output_embedding


def Featurize(df):
    # dummies = {'process': 0, 'flow': 1, 'file': 2, 'module': 3}
    dummies = {'process': 0, 'flow': 1, 'file': 2}

    nodes = {}
    labels = {}
    lblmap = {}
    neimap = {}
    edges = []

    for index, row in df.iterrows():
        actor_id, object_id = row['src_id'], row["dest_id"]
        object_type = row['object']

        if actor_id not in nodes:
            nodes[actor_id] = row['actorname']
        if object_id not in nodes:
            nodes[object_id] = row['objectname']

        # nodes.setdefault(actor_id, []).extend(row['phrase'])
        # nodes.setdefault(object_id, []).extend(row['phrase'])

        labels[actor_id] = dummies.get('process', -1)
        labels[object_id] = dummies.get(object_type, -1)

        if actor_id not in lblmap:
            lblmap[actor_id] = row['actorname']
        if object_id not in lblmap:
            lblmap[object_id] = row['objectname']

        neimap.setdefault(actor_id, set()).add(row['objectname'])
        neimap.setdefault(object_id, set()).add(row['actorname'])

        edge_type = row['action']
        edges.append((actor_id, object_id, edge_type))

    features, feat_labels, edge_index = [], [], [[], []]
    node_index = {}

    for node, nodename in nodes.items():
        features.append(infer(nodename))
        feat_labels.append(labels[node])
        node_index[node] = len(features) - 1

    for src, dst, action in edges:
        if action=='read' or action=='inbound':
            edge_index[0].append(node_index[dst])
            edge_index[1].append(node_index[src])
        elif action=='create':
            edge_index[0].append(node_index[src])
            edge_index[1].append(node_index[dst])
            edge_index[0].append(node_index[dst])
            edge_index[1].append(node_index[src])
        else:
            edge_index[0].append(node_index[src])
            edge_index[1].append(node_index[dst])

    mapp = list(node_index.keys())

    return features, np.array(feat_labels), edge_index, mapp, lblmap, neimap

nodes,labels,edges,mapp,lbl,nemap = Featurize(df)

In [26]:
len(nodes),len(labels),len(mapp)

(3082, 3082, 3082)

In [27]:
# 定义一个字典，存储每个节点对应的特征向量
nodes_vec = {}
for i, j in zip(mapp, nodes):
    nodes_vec[i] = j

In [61]:
# 交换源节点和目标节点，因为信息汇聚是由源节点汇聚到目标节点
# edges[0], edges[1] = edges[1], edges[0]

In [16]:
# infer('dwm.exe')

In [94]:
df.to_csv('df_xxx.csv', index=False)

In [28]:
df[df['actorname']=='\device\harddiskvolume1\windows\system32\wbem\wmiapsrv.exe']

,action,actorID,hostname,id,object,objectID,pid,ppid,principal,properties,tid,timestamp,actorname,objectname,command_line,phrase,src_id,dest_id


## lowest_ancestor_find

In [29]:
df_process_lineage = df[(df['action'] == 'create') & (df['object'] == 'process')]

# 记录每个进程的父进程及其时间戳和 cmdline 信息
process_parents = {}

# 进程谱系树生成
import networkx as nx

g = nx.MultiDiGraph()
node_map = {}
# count_node = 0
for index, row in df_process_lineage.iterrows():
    src, source_name, dst, dstname, etype, object_type = row['src_id'], row["actorname"],  row['dest_id'], row['objectname'], row['action'], row['object']
    command_line = row['command_line']
    if src not in node_map:
        node_map[src] = src
        g.add_node(src, type="process", pname=source_name)
    
    if dst not in node_map:
        node_map[dst] = dst
        if source_name==dstname:
            g.add_node(dst, type=object_type, pname=dstname)
        else:
            g.add_node(dst, type=object_type, pname=dstname)
    
    if g.has_edge(node_map[dst], node_map[src]):
        pass
    else:
        if g.has_edge(node_map[src], node_map[dst]):
            pass
        elif etype=='create':
            g.add_edge(node_map[src], node_map[dst], type=etype, commandline=command_line)
        else:
            pass
    if dst not in process_parents:
        process_parents[dst] = []
    process_parents[dst].append((src, command_line))

def find_lowest_common_ancestor(graph, target_pid):
    """
    查找某个进程的具有多个子进程的最低祖先。
    :param graph: NetworkX图对象
    :param target_pid: 目标进程ID
    :return: 具有多个子进程的最低祖先进程ID
    """
    # 获取目标进程的父进程
    def get_parent(pid):
        parents = process_parents.get(pid, [])
        if parents:
            return parents[0][0]  # 假设每个进程有一个父进程
        return None
    
    # 遍历并查找具有多个子进程的父进程
    def has_multiple_children(pid):
        children = list(graph.successors(pid))
        return len(children) > 1  # 如果父进程有多个子进程
    
    # 从目标进程开始，逐步向上遍历父进程
    current_pid = target_pid
    
    while True:
        parent_pid = get_parent(current_pid)
        if parent_pid is None:
            return None  # 找不到父进程，说明已经是根节点
        if has_multiple_children(parent_pid):
            # return parent_pid
            return current_pid  # 找到具有多个子进程的最低祖先
        current_pid = parent_pid  # 向上查找父进程

# 存储每个进程的最低祖先
process_lca_dict = {}



# 遍历所有进程，查找其最低祖先并存储
for node in g.nodes:
    lca = find_lowest_common_ancestor(g, node)
    if lca is not None:
        process_lca_dict[node] = process_parents.get(lca, 'meiyou')[0][1]

In [27]:
# len(process_lca_dict)

## model

In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv

class GNNWithRootEmbedding(nn.Module):
    def __init__(self, num_features, process_lca_dict, node_id_to_hash, node_hash_to_vec):
        super(GNNWithRootEmbedding, self).__init__()
        
        self.node_hash_to_vec = node_hash_to_vec # 存储节点的uuid到其初始向量的映射，其中uuid是hash值
        self.node_id_to_hash = node_id_to_hash # 存储节点的batch.n_id到节点uuid的映射，其中batch.n_id的数值，例如1，2，3，其中uuid是hash值
        self.process_lca_dict = process_lca_dict  # 存储节点与其根节点ID的映射
        self.num_features = num_features  # 节点特征维度
        
        # 第一层图卷积，输入维度为特征 + 根节点特征维度
        self.conv1 = SAGEConv(num_features, 128, normalize=True)  # 用于邻居节点特征的聚合
        # 第二层图卷积
        self.conv2 = SAGEConv(128, 64, normalize=True)
        
        # 用于根节点特征的变换矩阵W2
        self.root_transform = nn.Linear(num_features, 128)
        # 用于根节点特征的变换矩阵W3
        self.root_transform2 = nn.Linear(128, 64)

        self.linear = nn.Linear(64, 3)
    
    def root_embedding(self, node_ids):
        """
        通过根节点映射字典，直接获取根节点的特征。
        :param node_ids: 当前批次中的节点ID
        :return: 返回每个节点对应的根节点嵌入特征
        """
        # 获取节点对应的hash值
        node_hash = [self.node_id_to_hash[node_id] for node_id in node_ids]
        # 获取节点对应的根节点特征（通过process_lca_dict获得根节点ID）
        root_ids = [self.process_lca_dict.get(node_id, "meiyougengjiedian") for node_id in node_hash]
        
        # 在这里你可以调用任意方法返回根节点的特征
        # 假设根节点特征向量已经通过某种方式计算并存储，返回对应的特征
        # get_origin_embedding 函数假设返回根节点的嵌入特征
        # return torch.stack([torch.tensor(self.node_hash_to_vec.get(root_id, torch.zeros(self.num_features))) for root_id in root_ids], dim=0)
        return torch.stack([torch.tensor(infer(root_id), dtype=torch.float) for root_id in root_ids], dim=0)

    def forward(self, x, edge_index, batch_node_ids):

        x = self.encode(x, edge_index, batch_node_ids)

        x = self.linear(x)

        return F.softmax(x, dim=1)
        
    def encode(self, x, edge_index, batch_node_ids):
        """
        :param x: 节点特征矩阵
        :param edge_index: 边索引
        :param batch_node_ids: 当前批次的节点ID
        """
        # 获取当前批次节点的根节点特征
        root_embeds = self.root_embedding(batch_node_ids)

        # 第一步，聚合邻居特征
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        # 第二步，计算根节点特征的影响，根节点特征通过线性变换 W2
        root_embeds_transformed = self.root_transform(root_embeds.to(device))
        root_embeds_transformed = F.relu(root_embeds_transformed)
        root_embeds_transformed = F.dropout(root_embeds_transformed, p=0.2, training=self.training)

        # 最后，邻居特征加上根节点变换后的特征
        x = x + root_embeds_transformed

        # 第二层图卷积
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)


        root_embeds_transformed2 = self.root_transform2(root_embeds_transformed)
        root_embeds_transformed2 = F.relu(root_embeds_transformed2)
        root_embeds_transformed2 = F.dropout(root_embeds_transformed2, p=0.2, training=self.training)

        x = x + root_embeds_transformed2

        return x

In [31]:
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss

model = GNNWithRootEmbedding(64, process_lca_dict, mapp, nodes_vec).to(device)
# if not gnnTrain:
#     model.load_state_dict(torch.load(gnn_weights))
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [32]:
from sklearn.utils import class_weight

l = np.array(labels)
class_weights = class_weight.compute_class_weight(class_weight = "balanced",classes = np.unique(l),y = l)
class_weights = torch.tensor(class_weights,dtype=torch.float).to(device)
criterion = CrossEntropyLoss(weight=class_weights,reduction='mean')

graph = Data(x=torch.tensor(nodes,dtype=torch.float).to(device),y=torch.tensor(labels,dtype=torch.long).to(device), edge_index=torch.tensor(edges,dtype=torch.long).to(device))

In [33]:
graph.num_nodes

3082

In [34]:
# 定义训练节点为process节点
train_mask = torch.zeros(graph.num_nodes, dtype=torch.bool) 
for i , j in enumerate(labels):
    if j == 0:
        train_mask[i] = True
graph.train_mask = train_mask
print(sum(train_mask))

tensor(415)


In [35]:
loader = NeighborLoader(graph, num_neighbors=[-1, -1], batch_size=5000, shuffle=True, input_nodes=graph.train_mask)

In [214]:
# for i in range(len(edges[0])):
#     if edges[1][i]==11752:
#         print(edges[0][i], edges[1][i])

In [28]:
# num_baatch_nid = 0
# for i,batch in enumerate(loader):
#     print(batch.n_id)
#     num_baatch_nid=num_baatch_nid+len(batch.n_id)
# print(num_baatch_nid)

In [21]:
# mapp[14457]

In [36]:
def evaluate_model(batch):
    model.eval()
    with torch.no_grad():
        predictions = model(batch.x, batch.edge_index, batch.n_id)[:batch.batch_size]
        pred_labels = predictions.argmax(dim=1)
        y = batch.y[:batch.batch_size]
        correct_predictions = int((pred_labels == y).sum())
    return correct_predictions

In [37]:
gnn_weights = "../trained_weights/optc/cmdline_1.pth"

In [38]:
model.load_state_dict(torch.load(gnn_weights))

<All keys matched successfully>

In [39]:
total_loss = total_correct = total_nodes = 0

for batch in loader:
    num_nodes = batch.batch_size
    total_nodes += num_nodes
    total_correct += evaluate_model(batch)
# average_loss = total_loss / total_nodes
accuracy = total_correct / total_nodes

print(f"Accuracy: {accuracy:.5f}")

Accuracy: 0.79518


In [40]:
model.eval()
for batch in loader:
    with torch.no_grad():
        predictions = model(batch.x, batch.edge_index, batch.n_id)[:batch.batch_size]
        pred_labels = predictions.argmax(dim=1)
        y = batch.y[:batch.batch_size]
        # print(pred_labels.shape)
        # print(batch.y.shape)
        incorrect_nodes  = pred_labels != y
        # 进程节点总数
        process_entity_ids = batch.n_id[:batch.batch_size]
        incorrect_node_ids = batch.n_id[:batch.batch_size][incorrect_nodes]

In [41]:
len(incorrect_node_ids)

85

In [42]:
identified_ids = {mapp[idx] for idx in incorrect_node_ids}

In [43]:
EntitySet = {mapp[idx] for idx in process_entity_ids}

In [44]:
len(EntitySet)

415

In [45]:
infer("\device\harddiskvolume1\program files\windows defender\msascuil.exe")

array([-0.24878041, -0.65074223,  0.12679043, -0.27608603,  0.26596567,
        0.74465847, -1.1652167 , -0.10145646, -0.05871333, -0.6460427 ,
       -0.40959063, -0.08361121,  0.12893833,  0.7157233 , -0.06383117,
       -1.8865948 ,  0.32812032,  0.79702544, -0.6732028 ,  0.40922505,
       -1.0817277 , -0.58569384, -0.04345483,  1.8132694 ,  1.1944191 ,
       -1.2019447 ,  0.3651169 ,  0.5973879 , -0.8027509 , -0.9812311 ,
       -0.23781146,  0.989497  ,  0.08859032, -0.03236021,  1.6436543 ,
       -0.04319137, -0.3099056 , -1.1767569 , -1.0238857 ,  1.1058747 ,
        0.7167202 , -0.84522694, -0.4021925 , -0.8237396 , -0.9515681 ,
       -1.428004  ,  0.12203208,  0.8289292 ,  1.8093696 , -1.3478756 ,
        1.4878677 ,  0.22715421,  0.59725326, -2.4360468 ,  0.83402216,
        1.2233227 ,  0.24958143, -1.6777705 ,  2.1119628 , -0.44707468,
        0.18488859,  0.85044724,  1.1924627 ,  0.7393965 ], dtype=float32)

In [46]:
for i in incorrect_node_ids:
    if lbl[mapp[i]] !="\device\harddiskvolume1\windows\system32\conhost.exe" and lbl[mapp[i]] !="\device\harddiskvolume1\windows\system32\svchost.exe":
        print(lbl[mapp[i]])

\device\harddiskvolume1\program files (x86)\google\chrome\application\chrome.exe
\device\harddiskvolume1\program files (x86)\google\chrome\application\43.0.2357.65\installer\chrmstp.exe
\device\harddiskvolume1\program files (x86)\google\chrome\application\chrome.exe
\device\harddiskvolume1\windows\temp\rad1ba52.tmp\biguwcmnsucig.exe
\device\harddiskvolume1\program files (x86)\google\chrome\application\chrome.exe
net.exe
\device\harddiskvolume1\windows\system32\lsass.exe
\device\harddiskvolume1\windows\temp\rad720a0.tmp\biguwcmnsucig.exe
\device\harddiskvolume1\program files (x86)\google\chrome\application\chrome.exe
\device\harddiskvolume1\windows\temp\rad1c3a2.tmp\biguwcmnsucig.exe
\device\harddiskvolume1\program files (x86)\google\chrome\application\chrome.exe
cmd.exe
\device\harddiskvolume1\users\dcoombes\appdata\local\temp\ckfgw.exe
\device\harddiskvolume1\program files (x86)\notepad++\updater\gup.exe
\device\harddiskvolume1\program files (x86)\google\chrome\application\chrome.exe


In [47]:
for i in incorrect_node_ids:
    if lbl[mapp[i]] !="\device\harddiskvolume1\windows\system32\conhost.exe" and lbl[mapp[i]] !="\device\harddiskvolume1\windows\system32\svchost.exe":
        print(mapp[i])

95c4fe1013f458cfdd12fe0ec68df6a4dab958beb7721c6ca98ae3594709c83f
3b38fe6c953506a9a2f856eafe6ad5b8443b17868006261d5f285c87a1fc0332
98ad7fb6e1f366231d1befa7e7ecf0fd2d3f58218308f22e9a7363f55051ef21
c7b520d5664a34491405cc0856892386b49462bcb68056aee92defa343fb1b10
d402b224cf2d5e7a463e3ecd3b78a01206ece285284083701d6e3f1ce6b5a02c
b38c8b50a4633175bc831bfffaf0378114fe44ca729e12e834f1818804c55a65
fb8e69a2bfa832777e551ec2d8fea54062dcce239fc99493621c90d5b3c3a26d
b507869a7f72aa7cce2530591cbcb2c4bc1637db20c48cbb94698f786ae01a01
f9e93e491cdf4c15f6aa1348fca9b0ea86664253b602d18f6fde00b5100320f9
e3fca32855c43128ac7e3d762416623987fab0bd846731c52a84adb58e00d06a
7881b7c9f4879f58bf8c1cf0ea154fc9c659579713f8e3a9c1b44b9c276abba5
95690741cc6c83dbb442ab892e492b8940ddd750dec21e3ece7230174efc6ac0
9360696b13febb9300224b769f4aed1896fbe5e8e193b7ce683cabefd5eb7c32
c13916de17a9235f723e98de0e53db6fe6778befae346a299f3de5e3e55eda8e
690fa3cafb0df1f752e998aae6efeb7a9ab0482be807c946d0080f0cc4d2d9b9
a9c2c47cdb0c800413f18ac23

In [48]:
df['object'].value_counts()

file       3833
process     384
flow        373
Name: object, dtype: int64

In [49]:
df.head()

,action,actorID,hostname,id,object,objectID,pid,ppid,principal,properties,tid,timestamp,actorname,objectname,command_line,phrase,src_id,dest_id
0,inbound,08aab37f-a4c3-4f29-9053-5c8ccf08a45f,SysClient0051.systemia.com,d4a7b339-3cc7-4c77-a0a9-d4453494fec2,flow,84fd6d3c-be3b-40f1-8574-448449a6192f,4,0,NT AUTHORITY\SYSTEM,"{'acuity_level': '1', 'dest_ip': '142.20.59.25...",-1,2019-09-25 09:30:01,system,142.20.59.255-138,NaN,"[system, inbound, 142.20.59.255-138]",255baa7d3044f87213df50991bc4999b83bcb83df8de4d...,e08a3e6d53b11995e9f290d42d78390414630b8723213a...
1,inbound,08aab37f-a4c3-4f29-9053-5c8ccf08a45f,SysClient0051.systemia.com,0fa773e5-41ef-4453-b35a-ebc8850f599c,flow,ae5210ca-1602-41c5-ac0a-b820fd6b5e65,4,0,NT AUTHORITY\SYSTEM,"{'acuity_level': '1', 'dest_ip': '10.50.255.25...",-1,2019-09-25 09:30:01,system,10.50.255.255-138,NaN,"[system, inbound, 10.50.255.255-138]",255baa7d3044f87213df50991bc4999b83bcb83df8de4d...,94db62338644dbea58bfa77a0e6f9c79a25d452d6e6123...
2,inbound,99480f4c-aea3-4ae8-ba4d-3aabdf270cb9,SysClient0051.systemia.com,b9998a3f-d8a2-4d50-b668-968cc598588e,flow,e6ce56f2-30b1-4b2e-8f96-e6f6b19f851f,2192,2008,NT AUTHORITY\SYSTEM,"{'acuity_level': '1', 'dest_ip': '225.0.0.1', ...",-1,2019-09-25 09:30:01,\device\harddiskvolume1\python27\python.exe,225.0.0.1-5000,NaN,"[\device\harddiskvolume1\python27\python.exe, ...",7c865490e119e47808408788ab23557b89d06ea73c507e...,2e5184a3ab6979f2edb0508f3e863f7cb8e52a5a538247...
3,write,08aab37f-a4c3-4f29-9053-5c8ccf08a45f,SysClient0051.systemia.com,afe07e32-e2e0-4c9f-9ec9-c75e481e1b54,file,34562e6e-1e06-43c4-be26-05324c1e1734,4,0,NT AUTHORITY\SYSTEM,"{'acuity_level': '3', 'file_path': '\Device\Ha...",356,2019-09-25 09:30:02,system,\device\harddiskvolume1\windows\system32\confi...,NaN,"[system, write, \device\harddiskvolume1\window...",255baa7d3044f87213df50991bc4999b83bcb83df8de4d...,c03387058d8365b061b6f2c8978d06e35829ab5021f95e...
4,write,08aab37f-a4c3-4f29-9053-5c8ccf08a45f,SysClient0051.systemia.com,87be955a-ed76-4eed-9be3-c9e8979bb023,file,34d7e7b6-bcfa-471b-9209-feeee56be484,4,0,NT AUTHORITY\SYSTEM,"{'acuity_level': '3', 'file_path': '\Device\Ha...",356,2019-09-25 09:30:02,system,\device\harddiskvolume1\windows,NaN,"[system, write, \device\harddiskvolume1\windows]",255baa7d3044f87213df50991bc4999b83bcb83df8de4d...,afe0b1302db9cd78a33cacd9addb593f4a4a3899d0f4ba...


## evaluate

In [50]:
def load_ground_truth(gt_file):
    with open(gt_file, 'r') as file:
        gt_nodes = set(file.read().split())
    return gt_nodes

In [51]:
gt_nodes = load_ground_truth('../data/optc/attack3_groundtruth.txt')

In [28]:
# # 假设 gt_nodes 是一个包含 event_src 和 event_dest 值的集合
# # gt_nodes = [...]  # 示例集合

# # 用来存储对应的 actorid 和 objectid
# actorid_objectid_pairs = []

# # 遍历 gt_nodes 集合
# for n in gt_nodes:
#     # 筛选 df 中 event_src 或 event_dest 与 node 匹配的行
#     matching_rows = df[(df['src_id'] == n)]
#     matching_rows2 = df[(df['dest_id'] == n)]
#     # 如果找到匹配的行，获取 actorid 和 objectid
#     if not matching_rows.empty:
#         actorid = matching_rows['actorID'].values[0]  # 只取第一个匹配的 actorid
#         # objectid = matching_rows['objectid'].values[0]  # 只取第一个匹配的 objectid
#         actorid_objectid_pairs.append(actorid)
#         continue
#     if not matching_rows.empty:
#         objectid = matching_rows['objectID'].values[0]  # 只取第一个匹配的 actorid
#         # objectid = matching_rows['objectid'].values[0]  # 只取第一个匹配的 objectid
#         actorid_objectid_pairs.append(objectid)
#     if matching_rows.empty:
#         print('aaa')

# # 打开文件并写入
# with open('/home/cch/TraceSentinel/data/optc/attack3_groundtruth_2025.txt', 'w') as file:
#     for item in actorid_objectid_pairs:
#         file.write(item + '\n')  # 每个元素写入新的一行    


In [52]:
melicious_name = []
for i in gt_nodes:
    melicious_name.append(lbl[i])

In [53]:
len(identified_ids), len(gt_nodes), len(EntitySet)

(85, 55, 415)

In [ ]:
def calculate_metrics(TP, FP, FN, TN):
    FPR = FP / (FP + TN) if FP + TN > 0 else 0
    TPR = TP / (TP + FN) if TP + FN > 0 else 0

    prec = TP / (TP + FP) if TP + FP > 0 else 0
    rec = TP / (TP + FN) if TP + FN > 0 else 0
    fscore = (2 * prec * rec) / (prec + rec) if prec + rec > 0 else 0

    return prec, rec, fscore, FPR, TPR

def Get_Adjacent(ids, mapp, edges, hops):
    if hops == 0:
        return set()
    
    neighbors = set()
    for edge in zip(edges[0], edges[1]):
        if any(mapp[node] in ids for node in edge):
            neighbors.update(mapp[node] for node in edge)

    if hops > 1:
        neighbors = neighbors.union(Get_Adjacent(neighbors, mapp, edges, hops - 1))
    
    return neighbors

def helper(MP, all_pids, GP):
    """
    输入:
    MP预测错误的节点id的集合
    all_pids所有节点id的集合
    GP是grundtruth的节点id的集合
    edges是边的合集
    mapp是节点id的合集
    """
    TP = MP.intersection(GP)
    FP = MP - GP
    FN = GP - MP
    TN = all_pids - (GP | MP)


    # two_hop_gp找到GP的1跳邻居节点
    two_hop_gp = Get_Adjacent(GP, mapp, edges, 1)
    # two_hop_tp找到TP的1跳邻居节点
    two_hop_tp = Get_Adjacent(TP, mapp, edges, 1)

    # two_hop_fp找到FP的1跳邻居节点
    two_hop_fp = Get_Adjacent(FP, mapp, edges, 1)
    for i in list(FP):
        a = set()
        a.add(i)
        one_hop_fp = Get_Adjacent(a, mapp, edges, 1)
        # print(a)
        for j in one_hop_fp:
            if lbl[j] in melicious_name:
                FP.discard(i)
    

    FPL = FP - two_hop_gp
    TPL = TP.union(FN.intersection(two_hop_tp))
    FN = FN - two_hop_tp

    TPl, FPl, FNl, TNl = len(TPL), len(FPL), len(FN), len(TN)



    # TPl, FPl, FNl, TNl = len(TP), len(FP), len(FN), len(TN)
    print(TPl, FPl, FNl, TNl)
    prec, rec, fscore, FPR, TPR = calculate_metrics(TPl, FPl, FNl, TNl)
    print(f"Precision: {round(prec, 2)}, Recall: {round(rec, 2)}, Fscore: {round(fscore, 2)}")
    return FP, FN

In [1]:
# FP, FN = helper(identified_ids, EntitySet , gt_nodes)

In [498]:
FP, FN = helper(identified_ids, EntitySet , gt_nodes)

55 26 0 322
Precision: 0.68, Recall: 1.0, Fscore: 0.81


In [499]:
for i in FN:
    print(lbl[i])

In [500]:
for i in FN:
    print(i)

In [501]:
for i in FP:
    print(lbl[i])

\device\harddiskvolume1\program files (x86)\google\chrome\application\chrome.exe
winlogon.exe
\device\harddiskvolume1\program files (x86)\google\chrome\application\chrome.exe
\device\harddiskvolume1\program files (x86)\google\chrome\application\chrome.exe
\device\harddiskvolume1\program files (x86)\google\chrome\application\chrome.exe
\device\harddiskvolume1\program files (x86)\google\chrome\application\chrome.exe
\device\harddiskvolume1\program files (x86)\google\chrome\application\chrome.exe
\device\harddiskvolume1\program files (x86)\google\chrome\application\chrome.exe
\device\harddiskvolume1\windows\system32\ie4uinit.exe
winlogon.exe
\device\harddiskvolume1\program files\vmware\vmware tools\vmtoolsd.exe
\device\harddiskvolume1\windows\system32\searchindexer.exe
\device\harddiskvolume1\program files (x86)\google\chrome\application\chrome.exe
\device\harddiskvolume1\program files (x86)\google\chrome\application\chrome.exe
\device\harddiskvolume1\program files (x86)\google\chrome\app

In [242]:
aaa = r'\device\harddiskvolume1\windows\system32\services.exe'
aaa

'\\device\\harddiskvolume1\\windows\\system32\\services.exe'

In [222]:
infer(r'ping -w 2000 -n 2 127.0.0.1')

array([-0.5366901 , -3.798642  , -3.338743  , -2.3470292 , -2.0944061 ,
        2.064454  , -2.7115285 ,  3.3929758 , -3.3605957 ,  0.27205622,
        3.8560777 ,  2.7587087 ,  4.9146457 ,  1.6263459 ,  0.11060484,
        4.22251   ,  1.7340206 , -0.76376534,  1.2832522 , -2.617836  ,
       -4.3114967 ,  0.6241467 ,  0.86878204, -4.138137  ,  3.3814154 ,
       -3.191125  ,  2.586047  , -2.813652  ,  2.045826  ,  0.1931843 ,
       -3.5460432 , -4.740588  ,  2.4413705 ,  2.9101076 , -2.2671273 ,
       -1.7575626 ,  3.1448123 ,  0.76803124,  1.7728425 ,  0.89907414,
       -0.71444064, -3.610199  , -3.6143627 ,  1.1748449 ,  3.4710326 ,
        0.59157026, -1.4684983 , -5.176331  ,  1.2193542 ,  2.6263385 ,
       -0.5202121 , -0.9109209 ,  2.484388  , -3.355661  ,  2.8218691 ,
       -1.9253618 ,  4.548392  , -1.9750817 ,  1.3709741 ,  1.2645805 ,
       -1.7170382 ,  1.2561159 ,  1.1078136 ,  2.9150882 ], dtype=float32)

In [480]:
for i in FP:
    print(i)

690fa3cafb0df1f752e998aae6efeb7a9ab0482be807c946d0080f0cc4d2d9b9
95c4fe1013f458cfdd12fe0ec68df6a4dab958beb7721c6ca98ae3594709c83f
a8f7b8250128b443ef61c9ae308a46f10ac21fcd6d95503e1b731a727ebec4ed
14b50691e3f7ce3b8ed5d76e19df3226f57f9cca6abbe3caf7f0975403d4483f
530d50eec177047d3ee979eb9eabf433a20caab26de85983c6b6d1bd84793d9f
2671504f547adf15e5f39837d3e1521d35b88b8f4f05dfbd6414d03b5cbde27e
73307db9da0d3fbbe658a6ca3f225a5e720128a995776c1e178f032e9bd3b732
de1074def0cf808731ba67a6a1fce5668356e589ed5b2d9f65e619d50c11107f
45e1c507ad85a24d4b5a02585eb5a868f6b6e0be84a05d806e585bb7d9ad709f
d402b224cf2d5e7a463e3ecd3b78a01206ece285284083701d6e3f1ce6b5a02c
1c9894720791f17231f00353d6296350aee53f2a368e87ee35dc478a9327edda
98ad7fb6e1f366231d1befa7e7ecf0fd2d3f58218308f22e9a7363f55051ef21
7881b7c9f4879f58bf8c1cf0ea154fc9c659579713f8e3a9c1b44b9c276abba5
f0c011d6364706233edf85449f3cde8f7ed0af300433a1f167a225acc5fa5c77
3b38fe6c953506a9a2f856eafe6ad5b8443b17868006261d5f285c87a1fc0332
d334010edc2dd4d2b7986cab6

In [49]:
df[df['src_id'] == "c65b737f1b427f8d20f09a0e6febbc5143a63db518d164407581d218835fa495"]

,action,actorID,hostname,id,object,objectID,pid,ppid,principal,properties,tid,timestamp,actorname,objectname,command_line,phrase,src_id,dest_id
